# Estimating Heritability and Testing SNP Association using Maximum Likelihoods of Variance Component Models

Authors: Sarah Ji, Janet Sinsheimer and Hua Zhou

We will use a variance component model (also called a linear mixed model) to estimate heritability of a trait and then test for association to specified markers. We are conducting here the equivalent of a replication or a candidate SNP approach.  Normally if we had no prior hypothesis regarding particular loci (candidate gene approach), we would be first be testing markers genomewide using a GWAS approach that can handle pedigree data appropriately (see later lesson).  In that case, we would probably be using a fast score test approach rather than by maximum likelihood. However, maximum likelihood provides more accurate inference and parameters estimates and can be used to refine interference after screening.  We will also use the variance component frame work with maximum likelihood tests when conducting Mendelian randomization with families so it's useful for us to understand how the Julia package, VarianceComponentModels.jl, works and how to load in large data sets into Julia using the package, SnpArrays.jl. 



## Data files

In the GAW19 dataset there are 849 individuals in 20 families and 8,348,674 loci located on the odd chromosomes. All of these individuals were genotyped and a fraction of them also have sequence data. The majority of the sequenced loci are imputed for the remaining individuals. The largest family contains 107 individuals, the smallest, 27.

For illustration purposes, we changed the names of the loci to fit Julia's style guide Julia Style Guide.

Genetic Analysis Workshop 19: methods and strategies for analyzing human sequence and gene expression data in extended families and unrelated individuals, Corinne D. EngelmanEmail author, Celia M. T. Greenwood, Julia N. Bailey, Rita M. Cantor, Jack W. KentJr, Inke R. König, Justo Lorenzo Bermejo, Phillip E. Melton, Stephanie A. Santorico, Arne Schillert, Ellen M. Wijsman, Jean W. MacCluer and Laura Almasy, BMC Proceedings201610(Suppl 7):19 https://doi.org/10.1186/s12919-016-0007-z

Genome-wide QTL and eQTL analyses using Mendel, Hua Zhou, Jin Zhou, Tao Hu, Eric M. Sobel, Kenneth Lange https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5133530/

Typing ";" brings lets us use unix commands. Before starting the analysis we first use the unix command "ls" to check that the files are in our working directory.   


In [1]:
;ls -l pedLMM07052018.fam SNPnamesLMM.in SNPdata.bed

-rw-------@ 1 huazhou  staff  1778267565 Mar 11 22:14 SNPdata.bed
-rw-------@ 1 huazhou  staff   252411294 Jul  6 04:52 SNPnamesLMM.in
-rw-------@ 1 huazhou  staff       34398 Jul  6 04:41 pedLMM07052018.fam


## Read in the data

Take a look at the first 10 lines of the pedigree file, pedLMM07052918.fam. The columns are comma separated. This file is in the classic Mendel format, Family Id, Person ID, Father ID, Mother Id, sex as F (female) or M (male), monozygotic twin indicator, simtrait1 and simtrait2.  The traits were simulated using the Julia program TraitSimulation.jl using genotyped and imputed variants to have a main effect from two snps, one on chromosome 1 and the other on chromosome 13 as well as an interaction between these two SNPs.  If you look at the file carefully you will note that we don't know how individuals within a pedigree are related.  That is o.k. when using variant component models because we can use the GRM as a proxy for the Kinship matrix. 

In [2]:
;head pedLMM07052018.fam

2,200001,,,M,,0.870472967,1.245755458
2,200002,,,F,,-1.686156772,0.435395909
2,200003,,,F,,0.666343843,-0.150266906
2,200004,,,F,,0.083925361,-0.724815477
2,200005,,,M,,0.786505745,0.830477935
2,200006,,,M,,1.563371074,0.172045237
2,200007,,,F,,1.088919384,-0.395792086
2,200008,,,F,,-1.151107782,2.11290715
2,200009,,,F,,-0.36535092,0.759092316
2,200012,,,M,,-0.630388002,1.81987199


Read in the pedigree file into an array.

In [3]:
# columns are: :famid, :id, :moid, :faid, :sex, :twin, :simtrait1,:simtrait2
pedLMM = readcsv("pedLMM07052018.fam", Any; header = false)

849×8 Array{Any,2}:
  2   200001  ""  ""  "M"  ""   0.870473    1.24576 
  2   200002  ""  ""  "F"  ""  -1.68616     0.435396
  2   200003  ""  ""  "F"  ""   0.666344   -0.150267
  2   200004  ""  ""  "F"  ""   0.0839254  -0.724815
  2   200005  ""  ""  "M"  ""   0.786506    0.830478
  2   200006  ""  ""  "M"  ""   1.56337     0.172045
  2   200007  ""  ""  "F"  ""   1.08892    -0.395792
  2   200008  ""  ""  "F"  ""  -1.15111     2.11291 
  2   200009  ""  ""  "F"  ""  -0.365351    0.759092
  2   200012  ""  ""  "M"  ""  -0.630388    1.81987 
  2   200013  ""  ""  "M"  ""  -0.277382    1.46866 
  2   200018  ""  ""  "M"  ""   1.34288     0.417684
  2   200023  ""  ""  "F"  ""   1.88722    -0.819818
  ⋮                        ⋮                        
 47  4701128  ""  ""  "F"  ""   1.76189     2.17118 
 47  4701129  ""  ""  "M"  ""   1.35565     0.151142
 47  4701130  ""  ""  "M"  ""  -0.0690388   2.2869  
 47  4701131  ""  ""  "F"  ""   0.419444    2.81945 
 47  4701132  ""  ""  "F" 

We don't need to retain the ids so we retrieve the two phenotype data and put them in an array Y.

In [4]:
simtrait1 = convert(Vector{Float64}, pedLMM[:, 7])
simtrait2 = convert(Vector{Float64}, pedLMM[:, 8])
Y = [simtrait1 simtrait2]

849×2 Array{Float64,2}:
  0.870473    1.24576 
 -1.68616     0.435396
  0.666344   -0.150267
  0.0839254  -0.724815
  0.786506    0.830478
  1.56337     0.172045
  1.08892    -0.395792
 -1.15111     2.11291 
 -0.365351    0.759092
 -0.630388    1.81987 
 -0.277382    1.46866 
  1.34288     0.417684
  1.88722    -0.819818
  ⋮                   
  1.76189     2.17118 
  1.35565     0.151142
 -0.0690388   2.2869  
  0.419444    2.81945 
  1.47836    -0.276582
 -0.274552   -0.453944
  0.177762   -0.350015
 -0.464458    5.33595 
  0.0336769   0.11002 
  1.26861    -0.2     
 -0.319195    1.37059 
  0.114271    6.70812 

We retrieve sex data coded as 0 (male) or 1 (female), which means male is the reference group.  You can change the 
code to sex = map(x -> strip(x) == "M"? 1.0 : 0.0,  pedLMM[:, 5]) if you want female to be the reference group. 

In [5]:
sex = map(x -> strip(x) == "F"? 1.0 : 0.0,  pedLMM[:, 5])

849-element Array{Float64,1}:
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 ⋮  
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0

Take a look at the first 10 lines of the SNP definition file before we read in into an array using a unix command.

In [6]:
;head SNPnamesLMM.in

1,c1_54490,0,54490,A,G
1,c1_55550,0,55550,T,A
1,c1_57033,0,57033,C,T
1,c1_57064,0,57064,A,G
1,c1_57818,0,57818,A,C
1,c1_58432,0,58432,C,T
1,c1_58448,0,58448,A,G
1,c1_58814,0,58814,A,G
1,c1_59492,0,59492,G,A
1,c1_60829,0,60829,T,C


Read in the SNP definition file into a Julia array, skipping the first 2 lines.

In [7]:
# columns are: :snpid, :chrom, :pos, :allele1, :allele2, :groupname
snpLMM = readcsv("SNPnamesLMM.in", Any; header = false)

8348674×6 Array{Any,2}:
  1  "c1_54490"      0     54490  "A"  "G"
  1  "c1_55550"      0     55550  "T"  "A"
  1  "c1_57033"      0     57033  "C"  "T"
  1  "c1_57064"      0     57064  "A"  "G"
  1  "c1_57818"      0     57818  "A"  "C"
  1  "c1_58432"      0     58432  "C"  "T"
  1  "c1_58448"      0     58448  "A"  "G"
  1  "c1_58814"      0     58814  "A"  "G"
  1  "c1_59492"      0     59492  "G"  "A"
  1  "c1_60829"      0     60829  "T"  "C"
  1  "c1_61462"      0     61462  "A"  "T"
  1  "c1_61920"      0     61920  "A"  "G"
  1  "c1_62162"      0     62162  "A"  "G"
  ⋮                                    ⋮  
 21  "c21_48098508"  0  48098508  "T"  "C"
 21  "c21_48098650"  0  48098650  "C"  "T"
 21  "c21_48099610"  0  48099610  "C"  "T"
 21  "c21_48099701"  0  48099701  "C"  "T"
 21  "c21_48099782"  0  48099782  "C"  "G"
 21  "c21_48099931"  0  48099931  "T"  "C"
 21  "c21_48100155"  0  48100155  "T"  "C"
 21  "c21_48100388"  0  48100388  "T"  "C"
 21  "c21_48100711"  0  481007

We don't need the relative position of the snps in this case so we just retrieve SNP IDs.

In [8]:
snpid = map(x -> strip(string(x)), snpLMM[:, 2])

8348674-element Array{SubString{String},1}:
 "c1_54490"    
 "c1_55550"    
 "c1_57033"    
 "c1_57064"    
 "c1_57818"    
 "c1_58432"    
 "c1_58448"    
 "c1_58814"    
 "c1_59492"    
 "c1_60829"    
 "c1_61462"    
 "c1_61920"    
 "c1_62162"    
 ⋮             
 "c21_48098508"
 "c21_48098650"
 "c21_48099610"
 "c21_48099701"
 "c21_48099782"
 "c21_48099931"
 "c21_48100155"
 "c21_48100388"
 "c21_48100711"
 "c21_48100861"
 "c21_48101335"
 "c21_48101690"

Read in the SNP binary file using the SnpArray.jl package.

In [9]:
using SnpArrays

snpbinLMM = SnpArray("SNPdata"; people = size(pedLMM, 1), snps = size(snpLMM, 1))

849×8348674 SnpArrays.SnpArray{2}:
 (true, true)   (false, true)   …  (false, true)   (true, true)
 (true, true)   (false, true)      (true, true)    (true, true)
 (true, true)   (false, true)      (true, true)    (true, true)
 (true, true)   (true, true)       (true, true)    (true, true)
 (true, true)   (true, true)       (false, true)   (true, true)
 (true, true)   (true, false)   …  (false, true)   (true, true)
 (true, true)   (true, true)       (false, true)   (true, true)
 (true, true)   (true, true)       (true, true)    (true, true)
 (true, true)   (false, true)      (true, true)    (true, true)
 (true, true)   (true, true)       (true, true)    (true, true)
 (true, true)   (false, false)  …  (false, true)   (true, true)
 (true, true)   (true, true)       (false, true)   (true, true)
 (true, true)   (true, false)      (true, true)    (true, true)
 ⋮                              ⋱                              
 (true, true)   (true, false)      (true, true)    (true, true)
 (tru

INFO: v1.0 BED file detected


### Filtering the variant data to improve the quality of the GRM

First we check what the minor allele frequencies are. We can see by checking the quantiles that many of the loci are invariant or rather rare. By default the GRM function uses only variants with minor allele frequencies greater than 0.01 but we want to impose additional restrictions so that the MAF >0.05 and the percent success rate is >98% to avoid potential biases

In [10]:
maf, minor_allele, missings_by_snp, missings_by_person = summarize(snpbinLMM)

([0.0142012, 0.403226, 0.0711679, 0.0709951, 0.0777643, 0.0511084, 0.0517241, 0.000589623, 0.00117786, 0.0733496  …  0.00647821, 0.00176678, 0.00471143, 0.000588928, 0.0241461, 0.0153121, 0.0223793, 0.00294464, 0.494693, 0.00176678], Bool[true, true, true, true, true, true, true, true, true, true  …  true, true, true, true, true, true, true, true, true, true], [4, 43, 27, 25, 26, 37, 37, 1, 0, 31  …  0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1192, 1428, 2284, 16264, 1917, 1224, 2342, 2909, 1569, 4783  …  2898, 3139, 3349, 5221, 6088, 15218, 10806, 11987, 588, 10930])

In [11]:
quantile(maf, [0.0 .25 .5 .75 1.0])

1×5 Array{Float64,2}:
 0.0  0.00294464  0.0100118  0.114252  0.5

In [12]:
#first we filter out snps with genotype success rates < 98% and get the snp id's of snps with MAF>0.98
snp_idx, _ = filter(snpbinLMM, 0.98)

(Bool[true, false, false, false, false, false, false, true, true, false  …  true, true, true, true, true, true, true, true, true, true], Bool[true, true, true, true, true, true, true, true, true, true  …  true, true, true, true, true, true, true, true, true, true])

In [13]:
#now we find the index of the common snps (MAF greater than or equal to 0.05) with success rates >0.98
common_index = snp_idx .& (0.05 .≤ maf);

In [14]:
# now we put these snps into an array for use with the GRM function. 
data_common = snpbinLMM[ : , common_index]

849×2757072 SnpArrays.SnpArray{2}:
 (true, true)   (true, true)  (true, true)  …  (true, true)    (false, true) 
 (true, true)   (true, true)  (true, true)     (false, true)   (true, true)  
 (true, true)   (true, true)  (true, true)     (false, true)   (true, true)  
 (true, false)  (true, true)  (true, true)     (false, true)   (true, true)  
 (true, true)   (true, true)  (true, true)     (false, true)   (false, true) 
 (false, true)  (true, true)  (true, true)  …  (false, true)   (false, true) 
 (true, true)   (true, true)  (true, true)     (true, true)    (false, true) 
 (false, true)  (true, true)  (true, true)     (false, true)   (true, true)  
 (true, true)   (true, true)  (true, true)     (false, false)  (true, true)  
 (false, true)  (true, true)  (true, true)     (true, true)    (true, true)  
 (true, true)   (true, true)  (true, true)  …  (true, true)    (false, true) 
 (true, true)   (true, true)  (true, true)     (true, true)    (false, true) 
 (false, true)  (true, true) 

## Kinship via Genetic Relationship Matrix (GRM)

Recall that in using variance components (linear mixed models) we need a measure of the relatedness among individuals. Under the GRM formulation, the estimate of the global kinship coefficient of individuals $i$ and $j$ is
$$ \widehat\Phi_{GRMij}^  = \frac{1}{2S} \sum_{k=1}^S \frac{(x_{ik} -2p_k)(x_{jk} - 2p_k)}{2 p_k (1-p_k)}$$,
where $k$ ranges over the selected $S$ SNPs, $p_k$ is the minor allele frequency of SNP $k$, and $x_{ik}$ is the number of minor alleles in individual $i$s genotype at SNP $k$.

## Calculate the GRM matrix

By default, `grm` excludes SNPs with maf < 0.01 but we will use only the common snps with good success rates. 

In [15]:
Φgrm = grm(data_common)

849×849 Array{Float64,2}:
  0.481911      0.00738618   -0.00761969   …  -0.0134622    -0.00824202 
  0.00738618    0.496348      0.00112594       8.40893e-5    0.00663853 
 -0.00761969    0.00112594    0.489401         0.0094265    -0.000608303
  0.00732816    0.00836948   -0.0016404       -0.00695838   -0.00392229 
 -0.0134106     0.00797252    0.00166333       0.016533      0.00667759 
  0.0184094     0.00926725    0.0031459    …  -0.00382417   -0.000760924
  0.000637105  -0.000872816   0.0119925       -0.00305508   -0.00106326 
 -0.0250148     0.000824629   0.00668872       0.014293      0.0112914  
 -0.011422     -0.00382575   -0.0014202        0.00974001    0.00882354 
 -0.00526456    0.000503694   0.00646038      -0.000112577  -0.00367339 
 -0.00593591   -0.00147094    0.00404083   …  -0.000272617  -0.00578835 
 -0.010822     -0.00621223    0.00487296       0.00734693    0.00505043 
 -0.00749813    0.00238692    0.00475381      -0.00257178    0.00267197 
  ⋮                      

## Fit the null variance component model

Recall that we are using a variance component model with simtrait1 as the outcome. Under the null hypothesis simtrait1 is associated with sex (as a fixed effect).  We also need to account for the relatedness among individuals.  To do that we include a random effect and use the GRM matrix to describe the covariation structure. 
    $$ Y_{2i} = \mu +\beta_{sex} sex_i + A_i + e_i$$ 
    $$ A_i \sim N(0,\sigma^2_a)$$ $$e_i \sim N(0,\sigma^2_e)$$
    $$ Cov(Y_{2i},Y_{2j})=2\Phi_{ij} \sigma^2_a + 1_{i = j}\sigma^2_e$$

In [16]:
using VarianceComponentModels
# Null data model has two variance components but no SNP fixed effects

# form data as VarianceComponentVariate matrix 
#change the next commands if you want to run trait 2 or both traits (Y)
X = [ones(length(simtrait1)) sex]
#X = [ones(length(simtrait2)) sex]
nulldata = VarianceComponentVariate(Y[:,1], X, (2Φgrm, eye(length(simtrait1))))
#nulldata = VarianceComponentVariate(Y[:,2], X, (2Φgrm, eye(length(simtrait2))))

INFO: Recompiling stale cache file /Users/huazhou/.julia/lib/v0.6/Ipopt.ji for module Ipopt.


VarianceComponentModels.VarianceComponentVariate{Float64,2,Array{Float64,1},Array{Float64,2},Array{Float64,2}}([0.870473, -1.68616, 0.666344, 0.0839254, 0.786506, 1.56337, 1.08892, -1.15111, -0.365351, -0.630388  …  -0.0690388, 0.419444, 1.47836, -0.274552, 0.177762, -0.464458, 0.0336769, 1.26861, -0.319195, 0.114271], [1.0 0.0; 1.0 1.0; … ; 1.0 0.0; 1.0 1.0], ([0.963822 0.0147724 … -0.0269244 -0.016484; 0.0147724 0.992695 … 0.000168179 0.0132771; … ; -0.0269244 0.000168179 … 0.999926 0.0316366; -0.016484 0.0132771 … 0.0316366 1.01016], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]))

When we run an alternative model with the additional effects of the SNPs, it can be helpful to start from our best estimates from the null model. Initialize the variance component model parameters.

In [17]:
nullmodel = VarianceComponentModel(nulldata)

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0; 0.0], ([1.0], [1.0]), Array{Float64}(0,2), Char[], Float64[], -Inf, Inf)

In [18]:
@time nulllogl, nullmodel, = fit_mle!(nullmodel, nulldata; algo = :FS)

(-1233.713328747652, VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.371876; 0.133857], ([0.546329], [0.640098]), Array{Float64}(0,2), Char[], Float64[], -Inf, Inf), ([0.095306], [0.0690717]), [0.00908324 -0.00482352; -0.00482352 0.0047709], [0.0489772; 0.0699751], [0.00239877 -0.0028376; -0.0028376 0.00489651])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [19]:
# null model log-likelihood for no SNP effects
nulllogl

-1233.713328747652

In [20]:
# null model mean effects - in this case a grand mean and a sex effect
nullmodel.B

2×1 Array{Float64,2}:
 0.371876
 0.133857

In [21]:
# null model additive genetic variance
nullmodel.Σ[1]

1×1 Array{Float64,2}:
 0.546329

In [22]:
# null model environmental variance
nullmodel.Σ[2]

1×1 Array{Float64,2}:
 0.640098

### Heritability 
Calculate the proportion of the variance that can be attributed to additive genetic effects, the narrow sense heritability.  We calculate it here without any SNPs included. 

In [23]:
her_null = nullmodel.Σ[1]/(nullmodel.Σ[1]+nullmodel.Σ[2])

1×1 Array{Float64,2}:
 0.460483

# Fit variance component model with the causal SNPs

## Processing the SNP data
These data were simulated under a scenario in which two snps have large main effects and they interact. First we find the indexes of the two SNPs. 

In [24]:
#ind_c1_1360580 = find(x -> x == "c1_1360580", snpid)[1]
# Use can change this SNP if you would like to assess another's snps effect on the trait, e.g.:
#ind_c13_56233373 = find(x -> x == "c13_56233373", snpid)[1]
#ind_c1_754121 = find(x -> x == "c1_754121", snpid)[1]
ind_c1_1235710 = find(x -> x == "c1_1235710", snpid)[1]
ind_c13_56233373 = find(x -> x == "c13_56233373", snpid)[1]


6377239

Now we convert the SNP data into 0, 1, or 2 copies of the minor allele and form the data for the interaction of the two SNPs. 

In [25]:
#snp_c1_1360580 = convert(Vector{Float64}, snpbinLMM[:, ind_c1_1360580])
#snp_c13_56233373 = convert(Vector{Float64}, snpbinLMM[:, ind_c13_56233373])
#snp_c1_754121 = convert(Vector{Float64}, snpbinLMM[:, ind_c1_754121])
snp_c1_1235710 = convert(Vector{Float64}, snpbinLMM[:, ind_c1_1235710])
snp_c13_56233373 = convert(Vector{Float64}, snpbinLMM[:, ind_c13_56233373])
interact = snp_c1_1235710 .* snp_c13_56233373

849-element Array{Float64,1}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

## Look at the effect of a Single SNP snp_c1_1235710

In [26]:
# form data as VarianceComponentVariate - put the data in a form that VarianceComponentModels can use
Xalt = [ones(length(simtrait1)) sex snp_c1_1235710]
#Xalt = [ones(length(simtrait1)) sex snp_c13_56233373]
altdata = VarianceComponentVariate(Y[:,1], Xalt, (2Φgrm, eye(length(simtrait1))))
#altdata = VarianceComponentVariate(Y[:,2], Xalt, (2Φgrm, eye(length(simtrait2))))

VarianceComponentModels.VarianceComponentVariate{Float64,2,Array{Float64,1},Array{Float64,2},Array{Float64,2}}([0.870473, -1.68616, 0.666344, 0.0839254, 0.786506, 1.56337, 1.08892, -1.15111, -0.365351, -0.630388  …  -0.0690388, 0.419444, 1.47836, -0.274552, 0.177762, -0.464458, 0.0336769, 1.26861, -0.319195, 0.114271], [1.0 0.0 2.0; 1.0 1.0 0.0; … ; 1.0 0.0 0.0; 1.0 1.0 0.0], ([0.963822 0.0147724 … -0.0269244 -0.016484; 0.0147724 0.992695 … 0.000168179 0.0132771; … ; -0.0269244 0.000168179 … 0.999926 0.0316366; -0.016484 0.0132771 … 0.0316366 1.01016], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]))

In [27]:
altmodel = VarianceComponentModel(altdata)

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0; 0.0; 0.0], ([1.0], [1.0]), Array{Float64}(0,3), Char[], Float64[], -Inf, Inf)

### Set the starting values for the maximum likelihood estimation
Use the null model estimates as start values for the alternative model.

In [28]:
altmodel.B[1:2, :] = nullmodel.B
altmodel.B

3×1 Array{Float64,2}:
 0.371876
 0.133857
 0.0     

In [29]:
copy!(altmodel.Σ[1], nullmodel.Σ[1])
copy!(altmodel.Σ[2], nullmodel.Σ[2])
altmodel.Σ

([0.546329], [0.640098])

In [30]:
@time altlogl1, altmodel, = fit_mle!(altmodel, altdata; algo = :FS)

(-1180.0477802327648, VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([-0.065253; 0.162041; 0.587593], ([0.431819], [0.597935]), Array{Float64}(0,3), Char[], Float64[], -Inf, Inf), ([0.0806371], [0.0610984]), [0.00650235 -0.00357829; -0.00357829 0.00373301], [0.0619169; 0.0659945; 0.0548158], [0.00383371 -0.00263217 -0.00223942; -0.00263217 0.00435528 0.00015113; -0.00223942 0.00015113 0.00300477])

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [31]:
# alt model log-likelihood for the single SNP, snp_c1_1235710
altlogl1

-1180.0477802327648

In [32]:
# alt model mean effects
altmodel.B

3×1 Array{Float64,2}:
 -0.065253
  0.162041
  0.587593

In [33]:
# alt model additive genetic variance
altmodel.Σ[1]

1×1 Array{Float64,2}:
 0.431819

In [34]:
# alt model environmental variance
altmodel.Σ[2]

1×1 Array{Float64,2}:
 0.597935

Notice that the additive genetic variance and the environmental variance have both decreased.

To test the significance of the SNP, we use LRT

In [35]:
using Distributions
LRT1=2(altlogl1 - nulllogl)

107.33109702977436

In [36]:
#change the degrees of freedom if running a bivariate outcome
pval_snp_c1_1235710 = ccdf(Chisq(1), LRT1)

3.7667740107217424e-25

SNP, c1_1235710, has a significant effect. 

## Check for an interaction.  
### First calculate the log likelihood for additive effects of the two snps c1_1235710 and c13_56233373 without the interaction
Repeat the steps we took above:

In [37]:
# form data as VarianceComponentVariate
Xalt2 = [ones(length(simtrait1)) sex snp_c1_1235710 snp_c13_56233373]
altdata2 = VarianceComponentVariate(Y[:,1], Xalt2, (2Φgrm, eye(length(simtrait1))))

VarianceComponentModels.VarianceComponentVariate{Float64,2,Array{Float64,1},Array{Float64,2},Array{Float64,2}}([0.870473, -1.68616, 0.666344, 0.0839254, 0.786506, 1.56337, 1.08892, -1.15111, -0.365351, -0.630388  …  -0.0690388, 0.419444, 1.47836, -0.274552, 0.177762, -0.464458, 0.0336769, 1.26861, -0.319195, 0.114271], [1.0 0.0 2.0 0.0; 1.0 1.0 0.0 0.0; … ; 1.0 0.0 0.0 0.0; 1.0 1.0 0.0 0.0], ([0.963822 0.0147724 … -0.0269244 -0.016484; 0.0147724 0.992695 … 0.000168179 0.0132771; … ; -0.0269244 0.000168179 … 0.999926 0.0316366; -0.016484 0.0132771 … 0.0316366 1.01016], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]))

In [38]:
altmodel2 = VarianceComponentModel(altdata2)

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0; 0.0; 0.0; 0.0], ([1.0], [1.0]), Array{Float64}(0,4), Char[], Float64[], -Inf, Inf)

In [39]:
altmodel2.B[1:2, :] = nullmodel.B
altmodel2.B

4×1 Array{Float64,2}:
 0.371876
 0.133857
 0.0     
 0.0     

In [40]:
copy!(altmodel2.Σ[1], nullmodel.Σ[1])
copy!(altmodel2.Σ[2], nullmodel.Σ[2])
altmodel2.Σ

([0.546329], [0.640098])

In [41]:
@time altlogl2, altmodel2, = fit_mle!(altmodel2, altdata2; algo = :FS)

(-1172.3348087930935, VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([-0.153326; 0.166454; 0.598326; 0.390986], ([0.3982], [0.605134]), Array{Float64}(0,4), Char[], Float64[], -Inf, Inf), ([0.0773512], [0.0600862]), [0.00598321 -0.0033584; -0.0033584 0.00361035], [0.0654447; 0.0655376; 0.0542119; 0.0986926], [0.00428301 -0.00262229 -0.00224891 -0.00220929; -0.00262229 0.00429518 0.000152568 0.00012016; -0.00224891 0.000152568 0.00293893 0.000280351; -0.00220929 0.00012016 0.000280351 0.00974023])

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [42]:
altlogl2

-1172.3348087930935

In [43]:
# alt model mean effects
altmodel2.B

4×1 Array{Float64,2}:
 -0.153326
  0.166454
  0.598326
  0.390986

In [44]:
# alt model additive variance
altmodel2.Σ[1]

1×1 Array{Float64,2}:
 0.3982

In [45]:
# alt model environmental variance
altmodel2.Σ[2]

1×1 Array{Float64,2}:
 0.605134

### Test whether the addition of the second SNP improves the model fit by comparing the loglikelihood with just snp c1_1235710 to the loglikelihood with both snp_c1_1235710 and snp_c13_56233373s

In [46]:
using Distributions
LRT2=2(altlogl2 - altlogl1)

15.425942879342529

In [47]:
#change the degrees of freedom if running a bivariate outcome
pval_snp_c13_56233373 = ccdf(Chisq(2), LRT2)

0.00044699129416948916

## Check for evidence of an interaction between the two SNPs

In [48]:
# form data as VarianceComponentVariate
Xalt3 = [ones(length(simtrait1)) sex snp_c13_56233373 snp_c1_1235710 interact]
altdata3 = VarianceComponentVariate(Y[:,1], Xalt3, (2Φgrm, eye(length(simtrait1))))

VarianceComponentModels.VarianceComponentVariate{Float64,2,Array{Float64,1},Array{Float64,2},Array{Float64,2}}([0.870473, -1.68616, 0.666344, 0.0839254, 0.786506, 1.56337, 1.08892, -1.15111, -0.365351, -0.630388  …  -0.0690388, 0.419444, 1.47836, -0.274552, 0.177762, -0.464458, 0.0336769, 1.26861, -0.319195, 0.114271], [1.0 0.0 … 2.0 0.0; 1.0 1.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0], ([0.963822 0.0147724 … -0.0269244 -0.016484; 0.0147724 0.992695 … 0.000168179 0.0132771; … ; -0.0269244 0.000168179 … 0.999926 0.0316366; -0.016484 0.0132771 … 0.0316366 1.01016], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]))

Use the results of the two snp additive model as the starting point for the interaction model

In [49]:
altmodel3 = VarianceComponentModel(altdata3)

VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([0.0; 0.0; … ; 0.0; 0.0], ([1.0], [1.0]), Array{Float64}(0,5), Char[], Float64[], -Inf, Inf)

In [50]:
altmodel3.B[1:4, :] = altmodel2.B
altmodel3.B

5×1 Array{Float64,2}:
 -0.153326
  0.166454
  0.598326
  0.390986
  0.0     

In [51]:
copy!(altmodel3.Σ[1], altmodel2.Σ[1])
copy!(altmodel3.Σ[2], altmodel2.Σ[2])
altmodel3.Σ

([0.3982], [0.605134])

In [52]:
@time altlogl3, altmodel3, = fit_mle!(altmodel3, altdata3; algo = :FS)

(-1167.9571457027407, VarianceComponentModels.VarianceComponentModel{Float64,2,Array{Float64,2},Array{Float64,2}}([-0.0989004; 0.157861; … ; 0.534164; 0.370651], ([0.402279], [0.593224]), Array{Float64}(0,5), Char[], Float64[], -Inf, Inf), ([0.0771439], [0.0594444]), [0.00595119 -0.00331915; -0.00331915 0.00353363], [0.0676099; 0.0652252; … ; 0.0581619; 0.124851], [0.00457109 -0.00264555 … -0.00262662 0.00229045; -0.00264555 0.00425432 … 0.000213904 -0.000364664; … ; -0.00262662 0.000213904 … 0.00338281 -0.0027003; 0.00229045 -0.000364664 … -0.0027003 0.0155878])

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [53]:
altlogl3

-1167.9571457027407

In [54]:
# alt model mean effects
altmodel3.B

5×1 Array{Float64,2}:
 -0.0989004
  0.157861 
  0.176031 
  0.534164 
  0.370651 

In [55]:
# alt model additive variance
altmodel3.Σ[1]

1×1 Array{Float64,2}:
 0.402279

In [56]:
# alt model environmental variance
altmodel3.Σ[2]

1×1 Array{Float64,2}:
 0.593224

Test whether the interaction improves the model fit over the effects of the two SNPs alone

In [57]:
using Distributions
LRT3=2(altlogl3 - altlogl2)

8.755326180705651

In [58]:
#change the degrees of freedom if running a bivariate outcome
pval_snp_interact = ccdf(Chisq(1), LRT3)

0.0030869915556650997

Residual Heritability. The proportion of additive genetic variation remaining after including the SNPs and their interaction in the model.  

In [59]:
# ignore if running a bivariate outcome
her_alt=altmodel3.Σ[1]/(altmodel3.Σ[1]+altmodel3.Σ[2])

1×1 Array{Float64,2}:
 0.404096

Portion of the genetic variation explained by the snp is a measure of the effect of the snp on a signal trait. Note that in this simulated example the SNP effect is very large indeed. 

In [60]:
add_proport=(nullmodel.Σ[1]-altmodel3.Σ[1])/nullmodel.Σ[1]

1×1 Array{Float64,2}:
 0.263668

Portion of total variation explained by the snp is an alterative to the above. Again, typically the effects are not nearly so large.  

In [61]:
pheno_proport=(nullmodel.Σ[1]+nullmodel.Σ[2]-altmodel3.Σ[1]-altmodel3.Σ[2])/(nullmodel.Σ[1]+nullmodel.Σ[2])

1×1 Array{Float64,2}:
 0.160923